<a href="https://colab.research.google.com/github/cathieG/N_Gram/blob/main/Extract_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install pydriller pandas javalang

In [9]:
import pandas as pd
!add-apt-repository ppa:git-core/ppa
!apt-get update
!apt-get install git
from pydriller import Repository
import os
import javalang

from google.colab import drive
drive.mount('/content/drive')  #save csv into your google drive so that you can run this from any computer

Josh_folder_name = "git-methods-extracted"
Cathy_folder_name = "Gen AI for Soft Dev"

folder_path = f"/content/drive/My Drive/{Cathy_folder_name}" # Change if needed

PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/git-core/ppa/ubuntu/ jammy main'
Description:
The most current stable version of Git for Ubuntu.

For release candidates, go to https://launchpad.net/~git-core/+archive/candidate .
More info: https://launchpad.net/~git-core/+archive/ubuntu/ppa
Adding repository.
Press [ENTER] to continue or Ctrl-c to cancel.Aborted.
^C
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 htt

In [10]:
df_res = pd.read_csv(os.path.join(folder_path,'results.csv'))
repoList = []
for idx,row in df_res.iterrows():
  repoList.append("https://www.github.com/{}".format(row['name']))
print(repoList[0:5])
print(len(repoList))

['https://www.github.com/commonsguy/cw-android', 'https://www.github.com/cyrilmottier/greendroid', 'https://www.github.com/tokudu/androidpushnotificationsdemo', 'https://www.github.com/johannilsson/android-actionbar', 'https://www.github.com/johannilsson/android-pulltorefresh']
10949


In [11]:
import csv
from javalang.parse import parse
from javalang.tree import MethodDeclaration

def extract_methods_from_java(code):
    """
    Extract methods from Java source code using javalang parser.

    Args:
        code (str): The Java source code.

    Returns:
        list: A list of tuples containing method names and their full source code.
    """
    methods = []
    try:
        # Parse the code into an Abstract Syntax Tree (AST)
        tree = javalang.parse.parse(code)
        lines = code.splitlines()

        # Traverse the tree to find method declarations
        for _, node in tree.filter(javalang.tree.MethodDeclaration):
            method_name = node.name

            # Determine the start and end lines of the method
            start_line = node.position.line - 1
            end_line = None

            # Use the body of the method to determine its end position
            if node.body:
                last_statement = node.body[-1]
                if hasattr(last_statement, 'position') and last_statement.position:
                    end_line = last_statement.position.line

            # Extract method code
            if end_line:
                method_code = "\n".join(lines[start_line:end_line+1])
            else:
                # If end_line couldn't be determined, extract up to the end of the file
                method_code = "\n".join(lines[start_line:])

            methods.append((method_name, method_code))

    except Exception as e:
        print(f"Error parsing Java code: {e}")
    return methods


In [19]:
from git.exc import GitCommandError

def extract_methods_to_csv(repo_path, output_csv):
    """
    Extract methods from Java files in a repository and save them in a CSV file.

    Args:
        repo_path (str): Path to the Git repository.
        output_csv (str): Path to the output CSV file.
    """
    with open(output_csv, mode='w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Branch Name", "Commit Hash", "File Name", "Method Name", "Method Code", "Commit Link"])

        branch_name = "master"
        try:
            for commit in Repository(repo_path, only_in_branch=branch_name).traverse_commits():
                print(f"Processing commit: {commit.hash}")

                for modified_file in commit.modified_files:
                    if modified_file.filename.endswith(".java") and modified_file.source_code:
                        methods = extract_methods_from_java(modified_file.source_code)

                        for method_name, method_code in methods:
                            commit_link = f"{repo_path}/commit/{commit.hash}"
                            csv_writer.writerow([branch_name, commit.hash, modified_file.filename, method_name, method_code, commit_link])

                        print(f"Extracted methods from {modified_file.filename} in commit {commit.hash}")

        except GitCommandError as gce:
            print(f"Git error in repo {repo_path}: {gce}. Skipping this repository.")
            return  # Exit function for this repository and allow the next repo to be processed
        except Exception as e:
            print(f"Unexpected error in repo {repo_path}: {e}. Skipping this repository.")
            return


In [ ]:
counter = 49
for repo in repoList[counter:]:

    fileNameToSave = ''.join(repo.split('github.com')[1:])
    fileNameToSave = fileNameToSave.replace('/','_')

    # Specify the path to the output CSV file
    output_csv_file = os.path.join(folder_path, "extracted_methods_{}.csv".format(fileNameToSave))
    # Run the extraction
    extract_methods_to_csv(repo, output_csv_file)
    counter += 1

    print(f"-------------Processing {counter} repo: {repo}--------------")

Streaming output truncated to the last 5000 lines.
Processing commit: 23b9080af339fbf9258cbc5ee621630f9f4c99e3
Processing commit: c243e504ef4fcefe844ae7f4c53fbcb244494361
Processing commit: 49f4bcc2ab1640f32bb8f9cbfbc3f97211684d5d
Processing commit: 6317c0f9ba4ed451f9ab659a3f3c348d32495230
Processing commit: b4356fbed2744a6b3ef3c6838cb1e305746ce82e
Processing commit: 09a4693bdf396cf9089c9de3c7b96444c06208b8
Processing commit: 7d81a2b9c002fd67e75fd765a5656270e3d5aac2
Processing commit: 931908f67b6e47edabf3611dce7a777d13cbe9ec
Processing commit: 407d570470211e9a8e2713ab2ad9c0fca83723e9
Processing commit: 003dd932ec5ae8aaa4ee086f5cccfb34ebc1e939
Processing commit: 51d265a7bd4175bdc1c3bfc36c9fdc7d0d876c0a
Processing commit: c3774626028403d6666d4de17924537090abf6a2
Processing commit: 191905da9a79675f0ed019b432982fce0b7b29c0
Processing commit: 0a54cfedcb01aac8924224c2f16015563dde22ce
Processing commit: 75a72abbc3215af13b471a3de306107af39e7748
Processing commit: 22e58189accb4215de2f0159ffde9b